In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset=dataset.drop("User ID",axis=1)

In [5]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [7]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset[["Purchased"]]

In [8]:
indep.shape

(400, 3)

In [9]:
dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
from sklearn.svm import SVC

In [13]:
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['linear','rbf','poly','sigmoid'],'gamma':['auto','scale'],'C':[10,100,1000,2000,3000]}
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   33.9s finished
C:\Users\sony\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\sony\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_weighted', verbose=3)

In [14]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [16]:
print("The Confusion Matrix:",cm)

The Confusion Matrix: [[77  8]
 [ 4 45]]


In [17]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [18]:
print("The Report:",clf_report)

The Report:               precision    recall  f1-score   support

           0       0.95      0.91      0.93        85
           1       0.85      0.92      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.91       134
weighted avg       0.91      0.91      0.91       134



In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9618247298919568

In [20]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.023193,0.006398,0.004000,8.341244e-07,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.753889,0.665930,0.943041,0.899385,0.807042,0.100426,21
1,0.013595,0.001959,0.004000,1.393475e-06,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.888889,0.850543,0.944161,0.941902,0.898278,0.038157,1
2,0.021595,0.004080,0.003201,1.600315e-03,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.799620,0.812243,0.766556,0.943699,0.899385,0.843805,0.066180,13
3,0.013595,0.003199,0.003199,1.599479e-03,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.762677,0.743410,0.655795,0.738451,0.801015,0.740137,0.047410,36
4,0.020794,0.001600,0.002399,1.959022e-03,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.776290,0.753889,0.665930,0.943041,0.899385,0.807042,0.100426,21
5,0.019194,0.006879,0.001600,1.958983e-03,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.867478,0.888889,0.850543,0.944161,0.941902,0.898278,0.038157,1
6,0.020793,0.003918,0.003199,1.599717e-03,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.799620,0.812243,0.766556,0.943699,0.899385,0.843805,0.066180,13
7,0.017594,0.009663,0.002400,1.959216e-03,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.762677,0.743410,0.655795,0.738451,0.801015,0.740137,0.047410,36
8,0.095971,0.009120,0.003199,1.599741e-03,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.753889,0.665930,0.943041,0.899385,0.807042,0.100426,21
9,0.023193,0.006399,0.003999,3.162980e-07,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.888889,0.832483,0.944161,0.941902,0.894680,0.043030,3
